In [10]:
import requests
import json
from base64 import b64encode

In [2]:
client_id = "e291792abd4b4a1db646f561c823beb9"
client_secret = "26e6df1ae3614825940739dd03c31feb"

auth_url = 'https://accounts.spotify.com/api/token'
auth_header = 'Basic ' + b64encode((client_id + ':' + client_secret).encode()).decode()
url = "https://api.spotify.com/v1/search"

auth_data = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post(auth_url, headers={'Authorization': auth_header}, data=auth_data)

if auth_response.status_code == 200:
    token = auth_response.json().get('access_token')
    print('Token:', token)
else:
    print('Error:', auth_response.status_code)
    print(auth_response.text)





NameError: name 'b64encode' is not defined

In [1]:
token = "BQDLS51gIeE-tdAjx3S78yTN_8nus2WYiwXYS_tQE6ktSSIp2A-DaB8pwY-7Gm6iRnXOfkfGM3LmkB6LU-j07Vm3TRvAYmUUgVdAie09YVmUqwqrUtevOQAfvuBkF74RJQGnnECTNbs"

#artist_name = "Taylor Swift"

def get_spotify_id(artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1  
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        results = response.json()
        if results["artists"]["items"]:
            artist = results["artists"]["items"][0]
            id = artist["id"]
            return id
        else:
            return "No artist found."
    else:
        return f"Error: {response.status_code} - {response.text}"